In [1]:
%pip install ..

Processing /home/sjoshi/spuco
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spuco: filename=spuco-0.0.1-py3-none-any.whl size=49869 sha256=0f0edb27e40f8afc64da00836dfcaaf43d885bb73f2f96433de9a5ceb03ade12
  Stored in directory: /tmp/pip-ephem-wheel-cache-wy_hmlj7/wheels/ef/5d/43/a265894b1d52121a51705a208277e8d9a9670e95fa1a2e7ae6
Successfully built spuco
  Attempting uninstall: spuco
    Found existing installation: spuco 0.0.1
    Uninstalling spuco-0.0.1:
      Successfully uninstalled spuco-0.0.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch 

device = torch.device("cuda:7")

In [3]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_HARD

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

In [4]:
T.ToPILImage()(trainset[1000][0]).resize((28,28))

In [5]:
from spuco.utils import GroupLabeledDataset

group_labeled_trainset = GroupLabeledDataset(trainset, trainset.group_partition)

In [6]:
from spuco.models import model_factory 

model = model_factory("mlp", trainset[0][0].shape, trainset.num_classes).to(device)

In [8]:
from torch.optim import SGD
from spuco.invariant_train import GroupDRO

group_dro = GroupDRO(
    model=model,
    num_epochs=1,
    trainset=group_labeled_trainset,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_dro.train()

In [10]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Group (0, 0) Test Accuracy: 87.47044917257683
Group (0, 1) Test Accuracy: 82.74231678486997
Group (0, 2) Test Accuracy: 76.59574468085107
Group (0, 3) Test Accuracy: 82.50591016548464
Group (0, 4) Test Accuracy: 86.05200945626477
Group (1, 0) Test Accuracy: 83.12958435207824
Group (1, 1) Test Accuracy: 81.41809290953545
Group (1, 2) Test Accuracy: 76.96078431372548
Group (1, 3) Test Accuracy: 79.90196078431373
Group (1, 4) Test Accuracy: 82.1078431372549
Group (2, 0) Test Accuracy: 77.06666666666666
Group (2, 1) Test Accuracy: 84.8
Group (2, 2) Test Accuracy: 81.33333333333333
Group (2, 3) Test Accuracy: 72.8
Group (2, 4) Test Accuracy: 67.6470588235294
Group (3, 0) Test Accuracy: 82.41206030150754
Group (3, 1) Test Accuracy: 82.61964735516372
Group (3, 2) Test Accuracy: 84.63476070528968
Group (3, 3) Test Accuracy: 84.63476070528968
Group (3, 4) Test Accuracy: 84.38287153652394
Group (4, 0) Test Accuracy: 84.63476070528968
Group (4, 1) Test Accuracy: 79.34508816120906
Group (4, 2) Tes

{(0, 0): 87.47044917257683,
 (0, 1): 82.74231678486997,
 (0, 2): 76.59574468085107,
 (0, 3): 82.50591016548464,
 (0, 4): 86.05200945626477,
 (1, 0): 83.12958435207824,
 (1, 1): 81.41809290953545,
 (1, 2): 76.96078431372548,
 (1, 3): 79.90196078431373,
 (1, 4): 82.1078431372549,
 (2, 0): 77.06666666666666,
 (2, 1): 84.8,
 (2, 2): 81.33333333333333,
 (2, 3): 72.8,
 (2, 4): 67.6470588235294,
 (3, 0): 82.41206030150754,
 (3, 1): 82.61964735516372,
 (3, 2): 84.63476070528968,
 (3, 3): 84.63476070528968,
 (3, 4): 84.38287153652394,
 (4, 0): 84.63476070528968,
 (4, 1): 79.34508816120906,
 (4, 2): 84.1309823677582,
 (4, 3): 89.64646464646465,
 (4, 4): 89.39393939393939}

In [11]:
evaluator.worst_group_accuracy

((2, 4), 67.6470588235294)

In [12]:
evaluator.average_accuracy

84.88365187673114

In [13]:
evaluator.evaluate_spurious_task()

21.26